# Building a deep belief network

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm

from data import cicids
from utils import utils
from models import DBN

Check if GPU is active.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


### Create the DataLoader

In [3]:
# Get the datasets
train_data, val_data, test_data = cicids.get_balanced_dataset()

# How many instances have we got?
print('# instances in training set: ', len(train_data))
print('# instances in validation set: ', len(val_data))
print('# instances in testing set: ', len(test_data))

batch_size = 128

# Create the dataloaders - for training, validation and testing
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

# instances in training set:  1318783
# instances in validation set:  439595
# instances in testing set:  439595


### Instantiate the network, the loss function and the optimizer

In [4]:
# Defining some input variables
n_classes = 6
num_epochs = 3
tag = ""

# Creating a DBN
model = DBN(n_visible=47,
            n_hidden=(128, 64),
            k=(1, 1),
            learning_rate=(0.1, 0.1),
            momentum=(0, 0),
            decay=(0, 0), 
            batch_size=[64, 64],
            num_epochs=[1, 1],
            device=device)

# Training a DBN
model.fit(train_loader)

# Creating the optimzers
optimizer = [optim.Adam(m.parameters(), lr=0.001) for m in model.models]
optimizer.append(optim.Adam(model.fc.parameters(), lr=0.001))

In [ ]:
print(model)

### Train it

In [5]:
def train(
    model: torch.nn.ModuleList,
    optimizer: torch.optim,
    train_loader: torch.utils.data.DataLoader,
    valid_loader: torch.utils.data.DataLoader,
    num_epochs: int,
    device: torch.device,
    tag=''
):

    # Cross-Entropy loss is used for the discriminative fine-tuning
    criterion = nn.CrossEntropyLoss()

    history = {
        'train': {
            'total': 0,
            'loss': [],
            'accuracy': [],
            'output_pred': [],
            'output_true': []
        },
        'validation': {
            'total': 0,
            'loss': [],
            'accuracy': [],
            'output_pred': [],
            'output_true': []
        }
    }

    for epoch in range(1, num_epochs+1):

        ##################################
        ##          TRAIN LOOP          ##
        ##################################
        model.train()

        train_loss = 0.0
        train_steps = 0
        train_total = 0
        train_correct = 0

        train_output_pred = []
        train_output_true = []

        # For every possible batch
        print(f"{tag} Epoch {epoch}/{num_epochs}:")
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            for opt in optimizer:
                opt.zero_grad()

            # Passing the batch down the model
            outputs = model(inputs.float())

            # forward + backward + optimize
            loss = criterion(outputs, labels)
            loss.backward()
            
            # For every possible optimizer performs the gradient update
            for opt in optimizer:
                opt.step()

            train_loss += loss.cpu().item()
            train_steps += 1

            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            train_output_pred += outputs.argmax(1).tolist()
            train_output_true += labels.tolist()


        ############################################
        ##               VALID LOOP               ##
        ############################################
        model.eval()

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        val_total = 0
        val_correct = 0

        val_output_pred = []
        val_output_true = []

        for inputs, labels in valid_loader:
            with torch.no_grad():

                # Passing the batch down the model
                outputs = model(inputs.float())

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().item()
                val_steps += 1

                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

                val_output_pred += outputs.argmax(1).tolist()
                val_output_true += labels.tolist()

        history['train']['total'] = train_total
        history['train']['loss'].append(train_loss/train_steps)
        history['train']['accuracy'].append(train_correct/train_total)
        history['train']['output_pred'] = train_output_pred
        history['train']['output_true'] = train_output_true

        history['validation']['total'] = val_total
        history['validation']['loss'].append(val_loss/val_steps)
        history['validation']['accuracy'].append(val_correct/val_total)
        history['validation']['output_pred'] = val_output_pred
        history['validation']['output_true'] = val_output_true

        print(f'{tag} loss: {train_loss/train_steps} - acc: {train_correct/train_total} - val_loss: {val_loss/val_steps} - val_acc: {val_correct/val_total}\n')
    
    print(f"{tag} Finished Training")
    return history, train_loss/train_steps, train_correct/train_total, val_loss/val_steps, val_correct/val_total

In [ ]:
history, _, _, _, _ = train(model, optimizer, train_loader, valid_loader, num_epochs, device)

training_loss = history['train']['loss']
training_accuracy = history['train']['accuracy']
train_output_true = history['train']['output_true']
train_output_pred = history['train']['output_pred']

validation_loss = history['validation']['loss']
validation_accuracy = history['validation']['accuracy']
valid_output_true = history['validation']['output_true']
valid_output_pred = history['validation']['output_pred']

 Epoch 1/3:


100%|██████████| 10303/10303 [07:21<00:00, 23.31it/s]


 loss: 0.11694202192753587 - acc: 0.9676519943008062 - val_loss: 0.10293484276209401 - val_acc: 0.9608093813623904

 Epoch 2/3:


100%|██████████| 10303/10303 [07:53<00:00, 21.77it/s]


 loss: 0.03647624514655771 - acc: 0.9902948400153778 - val_loss: 0.07797997099692422 - val_acc: 0.9698631695082974

 Epoch 3/3:


100%|██████████| 10303/10303 [09:43<00:00, 17.66it/s]


### Plot loss vs iterations

In [ ]:
fig = plt.figure(figsize=(12, 8))
plt.plot(training_loss, label='train - loss')
plt.plot(validation_loss, label='validation - loss')
plt.title("Train and Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

fig = plt.figure(figsize=(12, 8))
plt.plot(training_accuracy, label='train - accuracy')
plt.plot(validation_accuracy, label='validation - accuracy')
plt.title("Train and Validation Accuracy")
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.legend(loc="best")
plt.show()

### Plot confusion matrix

In [ ]:
labels = ['Benign', 'Botnet ARES', 'Brute Force', 'DoS/DDoS', 'PortScan', 'Web Attack']

utils.plot_confusion_matrix(y_true=train_output_true,
                      y_pred=train_output_pred,
                      labels=labels,
                      title="Training Set - Normalized confusion matrix",
                      save=True,
                      filename="bal_dbn_train_confusion_matrix.png")

In [ ]:
from sklearn.metrics import classification_report

print("Training Set -- Classification Report", end="\n\n")
print(classification_report(train_output_true, train_output_pred, target_names=labels))

In [ ]:
utils.plot_confusion_matrix(y_true=valid_output_true,
                      y_pred=valid_output_pred,
                      labels=labels,
                      title="Validation Set - Normalized confusion matrix",
                      save=True,
                      filename="bal_dbn_valid_confusion_matrix.png")

In [ ]:
print("Validation Set -- Classification Report", end="\n\n")
print(classification_report(valid_output_true, valid_output_pred, target_names=labels))

### Test it

In [ ]:
def test(
    model: torch.nn.ModuleList,
    test_loader: torch.utils.data.DataLoader,
    device: torch.device,
):
    """Validate the network.

    Parameters
    ----------
    model: torch.nn.ModuleList
        Neural network model used in this example.

    test_loader: torch.utils.data.DataLoader
        DataLoader used in testing.

    device: torch.device
        (Default value = torch.device("cpu"))
        Device where the network will be trained within a client.

    Returns
    -------
        Tuple containing the history, and a detailed report.

    """

    model.eval()

    history = {
        'test': {
            'total': 0,
            'loss': 0.0,
            'accuracy': 0.0,
            'output_pred': [],
            'output_true': [],
            'output_pred_prob': []
        }
    }

    criterion = torch.nn.CrossEntropyLoss()

    test_loss = 0.0
    test_steps = 0
    test_total = 0
    test_correct = 0

    test_output_pred = []
    test_output_true = []
    test_output_pred_prob = []

    with torch.no_grad():
        for (inputs, labels) in tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs.float())
            
            loss = criterion(outputs, labels)

            test_loss += loss.cpu().item()
            test_steps += 1

            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

            test_output_pred += outputs.argmax(1).cpu().tolist()
            test_output_true += labels.tolist()
            test_output_pred_prob += nn.functional.softmax(outputs, dim=0).cpu().tolist()

    history['test']['total'] = test_total
    history['test']['loss'] = test_loss/test_steps
    history['test']['accuracy'] = test_correct/test_total
    history['test']['output_pred'] = test_output_pred
    history['test']['output_true'] = test_output_true
    history['test']['output_pred_prob'] = test_output_pred_prob

    print(f'Test loss: {test_loss/test_steps}, Test accuracy: {test_correct/test_total}')

    report = classification_report(
        y_true=history['test']['output_true'],
        y_pred=test_output_pred,
        zero_division=0,
        target_names=['Benign', 'Botnet ARES', 'Brute Force', 'DoS/DDoS', 'PortScan', 'Web Attack'],
        output_dict=True,
        digits=4,
    )

    return history, report

In [ ]:
#################
### TEST LOOP ###
#################
history, report = test(model, test_loader, device)

test_output_pred = history['test']['output_pred']
test_output_pred_prob = history['test']['output_pred_prob']

### Classification Report

In [ ]:
utils.plot_confusion_matrix(y_true=test_data.labels['label'].tolist(),
                            y_pred=test_output_pred,
                            labels=labels,
                            title="Testing Set - Normalized confusion matrix",
                            save=True,
                            filename="bal_dbn_test_confusion_matrix.png")

In [ ]:
print("Testing Set -- Classification Report", end="\n\n")
print(classification_report(test_data.labels['label'].tolist(), test_output_pred, target_names=labels))

### Plot ROC curve

In [ ]:
y_test = test_data.labels['label'].tolist()
y_test = pd.get_dummies(y_test).values
y_score = np.array(test_output_pred_prob)

In [ ]:
utils.plot_roc_curve(y_test=y_test,
                     y_score=y_score,
                     labels=labels,
                     save=True,
                     filename="bal_dbn_roc_curve.png")

### Plot Precision vs. Recall curve

In [ ]:
utils.plot_precision_recall_curve(y_test,
                                  y_score,
                                  labels=labels,
                                  save=True,
                                  filename="bal_dbn_prec_recall_curve.png")

### Save Model

In [ ]:
path = '../checkpoints/bal_deep_belief_network.pt'
torch.save({
            'epoch': num_epochs,
            'model_state_dict': model.state_dict(),
            # 'optimizer_state_dict': optimizer.state_dict(),
            }, path)